In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import psycopg2
import os
from string import Template

In [2]:
def create_con():
    con = (psycopg2.connect(user=os.environ.get("PGUSER"),        
                            password=os.environ.get("PGPASSWORD"),
                            host=os.environ.get("PGHOST"),
                            port="5439",
                            database="dev"))
    return con

In [3]:
from datetime import datetime, timedelta

In [4]:
date_format = "%Y-%m-%d"

def dt_str(date):
    """Convert Python datetime.datetime object to string YYYY-MM-DD."""
    return date.strftime(date_format)    

In [5]:
dt_start = dt_str(datetime.now() - timedelta(7))
dt_end = dt_str(datetime.now() - timedelta(1))
dates = {'dt_start' : dt_start, 'dt_end' : dt_end}

In [6]:
dates

{'dt_start': '2021-01-05', 'dt_end': '2021-01-11'}

### Rakuten Campaigns

#### advertiser_list for campaign converters

In [21]:
query = Template("""
    select
        sha256.email_sha256
    from rdn.reportableattributedconversion ac
        join rdn.session_emailsha256 sha256 on ac.sessionid = sha256.session_id
    where
        campaignid in (2584542651224484852, 2635354266114753803)
        and eventtime between '$dt_start' and '$dt_end'
        and conversion_classification = 'acquisition'
        and transactioneventtype = 'verifiedReferral'
    group by 1
""")

In [23]:
with create_con() as con:
    q = query.substitute(dates)
    data = pd.read_sql(q, con)

In [24]:
data

,email_sha256
0,S4WHbLYrrHy2I/zC6ChkAIm96JCz6TpYfX3Q2yJYp28=
1,fEtJqHj8LRuolX5GY6S13oBtrciQiRs37ej4So3kqAI=
2,agb1DsQFccgplFmvs1TPj+iIJFzFShtdkVqiE/vj+Vg=
3,5xsI8Kxt7rlBGDcUwHxtnotY6aXckGDPJJJNyZ2rpy4=
4,VR3Agxnb4TuSMF9K7NH1VWAO2LZ/35mJU8x1HA7atvc=
...,...
3766,n3OaPEQrL1pPLftC1EPIws1TevCcpIZEjfucshIJVqg=
3767,pbuukay1ldKCJ6jfv48sQGFqeKnbDSVF0/bTzSBjTHM=
3768,DH6m/MPnDyZGfBc+kvqGV96ohY863yS9tpuXLxE7T/4=
3769,b/24pXi4AaA/AyAxcHPxloII1xMbSL+sQV2PNsuVN5k=


#### Converter profile

In [25]:
query = Template(""" 
with  advertiser_list as (
        select
            sha256.email_sha256
        from rdn.reportableattributedconversion ac
            join rdn.session_emailsha256 sha256 on ac.sessionid = sha256.session_id
        where
            campaignid in (2584542651224484852, 2635354266114753803)
            and eventtime between '$dt_start' and '$dt_end'
            and conversion_classification = 'acquisition'
            and transactioneventtype = 'verifiedReferral'
        group by 1
    ),

-- If there is client high value user list, we need to merge them here.
advertiser_profile as (
        select
        s.email_sha256,
            -- negative values will be treated as missing by LiteGBM
            random() <= 0.2 as is_test,
            1 as user_level,
            calc_agegroup(age) as agegroup,
            case when lower(imp.gender) = 'm' then 1
                 when lower(imp.gender) = 'f' then 2
                 else 0
            end as gender,
            imp.partnerid as publisherid,
            case when lower(imp.device) = 'desktop' then 1
                 when lower(imp.device) = 'mobile' then 2
                 when lower(imp.device) = 'tablet' then 4
                 else 0
            end as deviceplatform,
            extract(hour from imp.eventtime) / 4 as hourofdaybucket,
            case when extract(dayofweek from imp.eventtime) in (1,2) then 1
                 when extract(dayofweek from imp.eventtime) in (3,4) then 2
                 else extract(dayofweek from imp.eventtime)
            end as dayofweekbucket
        from rdn.impression imp
            join rdn.session_emailsha256 sha256 on imp.sessionid = sha256.session_id
            join advertiser_list s on s.email_sha256 = sha256.email_sha256
        where
            eventtime between '$dt_start' and '$dt_end'
            -- all entries in a session will have the same attributes
            -- so only take the first to deduplicate
            and imp.rank = 1
    )
select * from advertiser_profile
"""
)

In [26]:
with create_con() as con:
    q = query.substitute(dates)
    data = pd.read_sql(q, con)

In [27]:
data

,email_sha256,is_test,user_level,agegroup,gender,publisherid,deviceplatform,hourofdaybucket,dayofweekbucket
0,vrGYBLFXb8ZcNZai8O5//Um7zVxF0ZwgPQbw38fBDDM=,False,1,4,1,243,1,0,6
1,lzA1VV4C1Kc4V7GbflGQ5nQvHLwIaQZHpCAUpOiotiE=,False,1,2,2,2820550498549280953,1,0,6
2,h1b3gi2YCLb382qw5rrGssQaXyniMRPBRj/4sGEbEs0=,False,1,5,2,299,1,0,6
3,7dELKtrpl+y6lkEKND9Fqjyu4W8IwI5FL9r8bTvBUe0=,False,1,2,2,2614530555266775655,2,0,6
4,EyLOYUUr/QQcZHM/N4LKgKQyqUCP5Rjne5VSjthF6Cg=,False,1,0,2,2045379072644613984,1,1,6
...,...,...,...,...,...,...,...,...,...
4887,pRLfy5nWtlAuNR9veFVvZi1J7ROno3/weaWO6ol1Xyw=,False,1,0,0,2671124901239497158,0,5,2
4888,drgwtPjHpWAVYtXpQx7O5O6z7CIlyzHmpO9GrOn+B78=,False,1,0,1,243,1,5,2
4889,QS0WhHeAJvQ4Idn08qzAvoiePzBlz4rr9AoDflzEMyE=,False,1,4,2,245,2,5,2
4890,8i0nL3DWizV1gPPBSOmKIrif88WQ6y7wCfuvuY6y3Qs=,False,1,0,2,2614530555266775655,2,5,2


In [28]:
data.describe()

,user_level,agegroup,gender,publisherid,deviceplatform,hourofdaybucket,dayofweekbucket
count,4892.0,4892.000000,4892.000000,4.892000e+03,4892.000000,4892.000000,4892.000000
mean,1.0,1.721995,1.364268,9.100184e+17,1.081971,3.159240,1.905969
std,0.0,2.372141,0.720480,1.195160e+18,0.382111,1.782199,1.835760
min,1.0,0.000000,0.000000,8.600000e+01,0.000000,0.000000,0.000000
25%,1.0,0.000000,1.000000,2.990000e+02,1.000000,2.000000,1.000000
50%,1.0,0.000000,2.000000,2.990000e+02,1.000000,4.000000,1.000000
75%,1.0,4.000000,2.000000,2.045379e+18,1.000000,5.000000,2.000000
max,1.0,6.000000,2.000000,2.830192e+18,4.000000,5.000000,6.000000


In [29]:
len(data['email_sha256'].unique())

3653

#### random_value_users

In [30]:
query = Template(""" 
with  advertiser_list as (
        select
            sha256.email_sha256
        from rdn.reportableattributedconversion ac
            join rdn.session_emailsha256 sha256 on ac.sessionid = sha256.session_id
        where
            campaignid in (2584542651224484852, 2635354266114753803)
            and eventtime between '$dt_start' and '$dt_end'
            and conversion_classification = 'acquisition'
            and transactioneventtype = 'verifiedReferral'
        group by 1
    ),

-- If there is client high value user list, we need to merge them here.
advertiser_profile as (
        select
        s.email_sha256,
            -- negative values will be treated as missing by LiteGBM
            random() <= 0.2 as is_test,
            1 as user_level,
            calc_agegroup(age) as agegroup,
            case when lower(imp.gender) = 'm' then 1
                 when lower(imp.gender) = 'f' then 2
                 else 0
            end as gender,
            imp.partnerid as publisherid,
            case when lower(imp.device) = 'desktop' then 1
                 when lower(imp.device) = 'mobile' then 2
                 when lower(imp.device) = 'tablet' then 4
                 else 0
            end as deviceplatform,
            extract(hour from imp.eventtime) / 4 as hourofdaybucket,
            case when extract(dayofweek from imp.eventtime) in (1,2) then 1
                 when extract(dayofweek from imp.eventtime) in (3,4) then 2
                 else extract(dayofweek from imp.eventtime)
            end as dayofweekbucket
        from rdn.impression imp
            join rdn.session_emailsha256 sha256 on imp.sessionid = sha256.session_id
            join advertiser_list s on s.email_sha256 = sha256.email_sha256
        where
            eventtime between '$dt_start' and '$dt_end'
            -- all entries in a session will have the same attributes
            -- so only take the first to deduplicate
            and imp.rank = 1
    ),

    random_value_user_profile as (
        -- Get a sample of rokt users from the target audience that are not in the seed list
        select
            random() <= 0.02 as is_test,
            0 as user_level,
            calc_agegroup(age) as agegroup,
            case when lower(imp.gender) = 'm' then 1
                 when lower(imp.gender) = 'f' then 2
                 else 0
            end as gender,
            imp.partnerid as publisherid,
            case when lower(imp.device) = 'desktop' then 1
                 when lower(imp.device) = 'mobile' then 2
                 when lower(imp.device) = 'tablet' then 4
                 else 0
            end as deviceplatform,
            extract(hour from imp.eventtime) / 4 as hourofdaybucket,
            case when extract(dayofweek from imp.eventtime) in (1,2) then 1
                 when extract(dayofweek from imp.eventtime) in (3,4) then 2
                 else extract(dayofweek from imp.eventtime)
            end as dayofweekbucket
        from
        rdn.impression imp
            join campaign c on c.campaignid = imp.campaignid
            join rdn.session_emailsha256 sha256 on imp.sessionid = sha256.session_id
            left join advertiser_profile ap on ap.email_sha256 = sha256.email_sha256
        where
            eventtime between '$dt_start' and '$dt_end'
            and imp.rank = 1
            and ap.email_sha256 is null
            order by random()
        limit 10000
    )
    select * from random_value_user_profile;
"""
)

In [31]:
with create_con() as con:
    q = query.substitute(dates)
    data = pd.read_sql(q, con)

In [32]:
data.columns

Index(['is_test', 'user_level', 'agegroup', 'gender', 'publisherid',
       'deviceplatform', 'hourofdaybucket', 'dayofweekbucket'],
      dtype='object')

In [33]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   is_test          10000 non-null  bool 
 1   user_level       10000 non-null  int64
 2   agegroup         10000 non-null  int64
 3   gender           10000 non-null  int64
 4   publisherid      10000 non-null  int64
 5   deviceplatform   10000 non-null  int64
 6   hourofdaybucket  10000 non-null  int64
 7   dayofweekbucket  10000 non-null  int64
dtypes: bool(1), int64(7)
memory usage: 556.8 KB


In [34]:
data.head()

,is_test,user_level,agegroup,gender,publisherid,deviceplatform,hourofdaybucket,dayofweekbucket
0,False,0,4,1,2215307220479662547,1,4,6
1,False,0,0,2,2760534150268376541,1,3,2
2,True,0,0,2,2074245483568304147,2,5,1
3,False,0,0,2,2754546188468645835,1,5,2
4,True,0,0,2,214,1,4,1


In [35]:
data.describe()

,user_level,agegroup,gender,publisherid,deviceplatform,hourofdaybucket,dayofweekbucket
count,10000.0,10000.000000,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000
mean,0.0,1.243500,1.18490,1.884461e+18,1.453800,2.731900,1.941200
std,0.0,2.007439,0.78433,1.153593e+18,0.599584,1.856493,1.842521
min,0.0,0.000000,0.00000,1.300000e+01,0.000000,0.000000,0.000000
25%,0.0,0.000000,1.00000,3.660000e+02,1.000000,1.000000,1.000000
50%,0.0,0.000000,1.00000,2.646162e+18,1.000000,3.000000,1.000000
75%,0.0,3.000000,2.00000,2.671125e+18,2.000000,4.000000,2.000000
max,0.0,8.000000,2.00000,2.830147e+18,4.000000,5.000000,6.000000


### Training dataset

In [7]:
query = Template(""" 
with  advertiser_list as (
        select
            sha256.email_sha256
        from rdn.reportableattributedconversion ac
            join rdn.session_emailsha256 sha256 on ac.sessionid = sha256.session_id
        where
            campaignid in (2584542651224484852, 2635354266114753803)
            and eventtime between '$dt_start' and '$dt_end'
            and conversion_classification = 'acquisition'
            and transactioneventtype = 'verifiedReferral'
        group by 1
    ),

-- If there is client high value user list, we need to merge them here.
advertiser_profile as (
        select s.email_sha256,
            -- negative values will be treated as missing by LiteGBM
            random() <= 0.2 as is_test,
            1 as user_level,
            calc_agegroup(age) as agegroup,
            case when lower(imp.gender) = 'm' then 1
                 when lower(imp.gender) = 'f' then 2
                 else 0
            end as gender,
            imp.partnerid as publisherid,
            case when lower(imp.device) = 'desktop' then 1
                 when lower(imp.device) = 'mobile' then 2
                 when lower(imp.device) = 'tablet' then 4
                 else 0
            end as deviceplatform,
            extract(hour from imp.eventtime) / 4 as hourofdaybucket,
            case when extract(dayofweek from imp.eventtime) in (1,2) then 1
                 when extract(dayofweek from imp.eventtime) in (3,4) then 2
                 else extract(dayofweek from imp.eventtime)
            end as dayofweekbucket
        from rdn.impression imp
            join rdn.session_emailsha256 sha256 on imp.sessionid = sha256.session_id
            join advertiser_list s on s.email_sha256 = sha256.email_sha256
        where
           eventtime between '$dt_start' and '$dt_end'
            -- all entries in a session will have the same attributes
            -- so only take the first to deduplicate
            and imp.rank = 1
    ),

    random_value_user_profile as (
        -- Get a sample of rokt users from the target audience that are not in the seed list
        select
            random() <= 0.02 as is_test,
            0 as user_level,
            calc_agegroup(age) as agegroup,
            case when lower(imp.gender) = 'm' then 1
                 when lower(imp.gender) = 'f' then 2
                 else 0
            end as gender,
            imp.partnerid as publisherid,
            case when lower(imp.device) = 'desktop' then 1
                 when lower(imp.device) = 'mobile' then 2
                 when lower(imp.device) = 'tablet' then 4
                 else 0
            end as deviceplatform,
            extract(hour from imp.eventtime) / 4 as hourofdaybucket,
            case when extract(dayofweek from imp.eventtime) in (1,2) then 1
                 when extract(dayofweek from imp.eventtime) in (3,4) then 2
                 else extract(dayofweek from imp.eventtime)
            end as dayofweekbucket
        from rdn.impression imp
            join campaign c on c.campaignid = imp.campaignid
            join rdn.session_emailsha256 sha256 on imp.sessionid = sha256.session_id
            left join advertiser_profile ap on ap.email_sha256 = sha256.email_sha256
        where
            eventtime between '$dt_start' and '$dt_end'
            and imp.rank = 1
            and ap.email_sha256 is null
            order by random()
        limit 10000
    )
    
    select
        is_test,
        user_level,
        agegroup,
        gender,
        publisherid,
        deviceplatform,
        hourofdaybucket,
        dayofweekbucket
    from
        advertiser_profile
    union all
    select
        is_test,
        user_level,
        agegroup,
        gender,
        publisherid,
        deviceplatform,
        hourofdaybucket,
        dayofweekbucket
    from
        random_value_user_profile
;
"""
)

In [8]:
with create_con() as con:
    q = query.substitute(dates)
    data = pd.read_sql(q, con)

In [9]:
data.head()

,is_test,user_level,agegroup,gender,publisherid,deviceplatform,hourofdaybucket,dayofweekbucket
0,False,1,0,2,2552816994824234651,1,4,0
1,False,1,6,1,367,1,1,1
2,False,1,3,2,2614530555266775655,2,4,2
3,False,1,3,2,2614530555266775655,2,2,1
4,False,1,3,2,2614530555266775655,2,4,1


In [10]:
data.to_csv("data.csv")

In [11]:
import numpy as np
import pandas as pd
import math
import random

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime, timedelta
import warnings; warnings.filterwarnings(action='once')

large = 22; med = 16; small = 12
params = {'axes.titlesize': large,
          'legend.fontsize': med,
          'figure.figsize': (16, 10),
          'axes.labelsize': med,
          'axes.titlesize': med,
          'xtick.labelsize': med,
          'ytick.labelsize': med,
          'figure.titlesize': large}

plt.rcParams.update(params)
plt.style.use('seaborn-whitegrid')
sns.set_style("white")
%matplotlib inline

import pandas_profiling
import datetime

In [13]:
import warnings
warnings.simplefilter('always')

data = pd.read_csv("data.csv")
data.head()

/Users/linghuang/miniconda3/envs/notebook/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Unnamed: 0,is_test,user_level,agegroup,gender,publisherid,deviceplatform,hourofdaybucket,dayofweekbucket
0,0,False,1,0,2,2552816994824234651,1,4,0
1,1,False,1,6,1,367,1,1,1
2,2,False,1,3,2,2614530555266775655,2,4,2
3,3,False,1,3,2,2614530555266775655,2,2,1
4,4,False,1,3,2,2614530555266775655,2,4,1


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14726 entries, 0 to 14725
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   Unnamed: 0       14726 non-null  int64
 1   is_test          14726 non-null  bool 
 2   user_level       14726 non-null  int64
 3   agegroup         14726 non-null  int64
 4   gender           14726 non-null  int64
 5   publisherid      14726 non-null  int64
 6   deviceplatform   14726 non-null  int64
 7   hourofdaybucket  14726 non-null  int64
 8   dayofweekbucket  14726 non-null  int64
dtypes: bool(1), int64(8)
memory usage: 934.9 KB


/Users/linghuang/miniconda3/envs/notebook/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
user_profile = pandas_profiling.ProfileReport(data)
user_profile.to_file(output_file='data.html')

/Users/linghuang/miniconda3/envs/notebook/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
